In [40]:
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
import pandas as pd
import requests
from us import states

from team import dataframe

In [42]:
df = dataframe.get_all_d1_schools()
dataframe.add_ap_rankings_to_dataframe(df)
display(df)
display(df.loc[df["School"] == "Gonzaga University"])

,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking
0,"University at Albany, SUNY",Great Danes,SEFCU Arena,America East,"(5) 2006, 2007, 2013, 2014, 2015",(0),(0),None
1,Binghamton University,Bearcats,Binghamton University Events Center,America East,(1) 2009,(0),(0),None
2,University of Hartford,Hawks,Chase Arena at Reich Family Pavilion,America East,(1) 2021,(0),(0),None
3,University of Maine,Black Bears,Alfond Arena,America East,(0),(0),(0),None
4,"University of Maryland, Baltimore County",Retrievers,Retriever Activities Center,America East,"(2) 2008, 2018",(0),(0),None
...,...,...,...,...,...,...,...,...
353,Seattle University,Redhawks,KeyArena,WAC,"(11) 1953, 1954, 1955, 1956, 1958, 1961, 1962,...",(1) 1958,(0),None
354,Stephen F. Austin State University,Lumberjacks,William R. Johnson Coliseum,WAC,"(5) 2009, 2014, 2015, 2016, 2018",(0),(0),None
355,Tarleton State University,Texans,Wisdom Gymnasium,WAC,(0),(0),(0),None
356,Utah Valley University,Wolverines,UCCU Center,WAC,(0),(0),(0),None


,School,Nickname,Home arena,Conference,Tournament appearances,Final Four appearances,Championship wins,AP Ranking
336,Gonzaga University,Bulldogs,McCarthey Athletic Center,West Coast,"(24) 1995, 1999, 2000, 2001, 2002, 2003, 2004,...","(2) 2017, 2021",(0),1
